In [2]:
using Random
Random.seed!(1234)

TaskLocalRNG()

In [3]:
using Distributions

In [4]:
using JuMP, MosekTools

# Function to get routes from solution

In [5]:
function get_routes(x)
    places = length(x[1,:])
    routes = []
    for j = 2:places
        if x[1,j]
            new_route = [1, j]
            next = j
            while next != 1
                for k = 1:places
                    if x[next, k]
                        append!(new_route, k)
                        next = k
                        break
                    end
                end
            end
            append!(routes, [new_route])
        end
    end
    routes
end

function get_costs(routes, c)
    costs = []
    for route in routes
        source = route[1]
        desti = route[2]
        new_cost = c[source, desti]
        for i = 3:length(route)
            source = desti
            desti = route[i]
            new_cost += c[source, desti]
        end
        append!(costs, new_cost)
    end
    costs
end

function print_routes_and_costs_helper(routes, costs)
    for r = 1:length(routes)
        displayed_route = map(x -> x-1, routes[r])
        println(displayed_route, "\t", costs[r])
    end
end        

function print_routes_and_costs(x, c)
    routes = get_routes(x)
    costs = get_costs(routes, c)
    print_routes_and_costs_helper(routes, costs)
end

print_routes_and_costs (generic function with 1 method)

# Inputs

### Setup:

1 depot (node 0), n=15 clients (nodes 1 to 15). <br>
M vehicles, each with capacity Q. <br>
client i has parcel with weight w[i]. <br>
c(i,j) is cost to get from node i to node j. <br>

In [6]:
n = 15;

In [7]:
M = 4;

In [8]:
Q = 1000;

In [9]:
w = rand(10:400, n);
w[2] = 495;
print(w)

[137, 495, 95, 359, 148, 164, 382, 321, 203, 302, 236, 294, 12, 87, 181]

In [10]:
sum(w)

3416

In [11]:
# c_mat starts from 1
c_mat = rand(Uniform(0.1, 40), n+1, n+1);
for i = 1:(n+1)
    c_mat[i,i] = 0;
end
c_mat = round.(c_mat, digits=1)

# c(,) starts from 0
function c(x,y)
    c_mat(x-1, y-1)
end

c (generic function with 1 method)

# Data prep

###### w_cor and c_mat are the two objects we want to use, with depot as node 1 and the clients as nodes 2:n+1 since then these align with the variable indices. (since indices start at 1 in Julia.)

In [12]:
w_cor = vcat([0], w)

16-element Vector{Int64}:
   0
 137
 495
  95
 359
 148
 164
 382
 321
 203
 302
 236
 294
  12
  87
 181

# VRP model

In [13]:
vrp = Model(Mosek.Optimizer)
@variable(vrp, x[1:n+1, 1:n+1], Bin)
@variable(vrp, s[1:n+1] >= 0)
@objective(vrp, Min, transpose(vec(c_mat))*vec(x))
for i = 1: n+1
    @constraint(vrp, x[i,i] == 0)
end
@constraint(vrp, sum(x[1, 2:n+1]) <= M)
@constraint(vrp, sum(x[2:n+1, 1]) <= M)
for i = 2:n+1
    @constraint(vrp, sum(x[i, 1:n+1]) == 1)
end
for j = 2:n+1
    @constraint(vrp, sum(x[1:n+1, j]) == 1)
end
@constraint(vrp, s[1] == 0)
for i = 2:n+1
    @constraint(vrp, s[i] <= Q)
end
for i = 1:n+1
    for j = 2:n+1
        @constraint(vrp, s[i] - s[j] + w_cor[j] <= (Q + w_cor[j]) * (1 - x[i,j]))
    end
end
print(vrp)

In [14]:
optimize!(vrp)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 304             
  Cones                  : 0               
  Scalar variables       : 272             
  Matrix variables       : 0               
  Integer variables      : 256             

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.02
Presolved problem: 255 variables, 257 constraints, 1140 non-zeros
Presolved problem: 0 general integer, 240 binary, 15 continuous
Clique table size: 30
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        NA                   9.0337817205e+01     NA          0.1   
0        1        1        0        2.2240000000e+02     9.0337817205e+01     59.38       0.2   
Cut generation started.
0        1        1        0        2

13450    10679    6558     11       1.4500000000e+02     1.1228990172e+02     22.56       6.7   
13604    10799    6632     13       1.4500000000e+02     1.1232798863e+02     22.53       6.7   
13766    10919    6692     18       1.4500000000e+02     1.1239416935e+02     22.49       6.8   
13924    11040    6742     12       1.4500000000e+02     1.1245675215e+02     22.44       6.9   
14081    11163    6805     20       1.4500000000e+02     1.1252069538e+02     22.40       6.9   
14237    11283    6867     23       1.4500000000e+02     1.1256149137e+02     22.37       7.0   
14399    11409    6941     19       1.4500000000e+02     1.1262528798e+02     22.33       7.1   
14562    11536    7002     14       1.4500000000e+02     1.1266543547e+02     22.30       7.1   
14721    11664    7075     20       1.4500000000e+02     1.1271500404e+02     22.27       7.2   
14877    11785    7141     16       1.4500000000e+02     1.1279035349e+02     22.21       7.3   
15042    11908    7212     26 

32566    24022    9300     19       1.2920000000e+02     1.1768185810e+02     8.91        13.7  
32774    24145    9290     18       1.2920000000e+02     1.1772619370e+02     8.88        13.8  
32985    24272    9275     17       1.2920000000e+02     1.1779549602e+02     8.83        13.9  
33225    24402    9237     14       1.2920000000e+02     1.1783003895e+02     8.80        13.9  
33421    24526    9211     14       1.2920000000e+02     1.1786996833e+02     8.77        14.0  
33621    24653    9191     16       1.2920000000e+02     1.1789357024e+02     8.75        14.1  
33845    24778    9165     19       1.2920000000e+02     1.1795904537e+02     8.70        14.1  
34074    24903    9118     16       1.2920000000e+02     1.1801487799e+02     8.66        14.2  
34281    25028    9109     16       1.2920000000e+02     1.1805543650e+02     8.63        14.3  
34499    25156    9097     13       1.2920000000e+02     1.1810269346e+02     8.59        14.3  
34707    25282    9093     22 

In [15]:
solution_summary(vrp)

MOSEK error 2950: No dual information is available for the integer solution.


* Solver : Mosek

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Message from the solver:
  "Mosek.MSK_SOL_STA_INTEGER_OPTIMAL"

* Candidate solution
  Objective value      : 129.2
  Objective bound      : 129.20000000000002

* Work counters
  Solve time (sec)   : 19.92597
  Simplex iterations : 388693
  Barrier iterations : 16
  Node count         : 57731


In [16]:
vrp_x = !=(0).(value.(x))

16×16 BitMatrix:
 0  0  0  0  0  0  0  1  1  0  1  1  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0

In [17]:
vrp_s = value.(s)

16-element Vector{Float64}:
    0.0
  439.0
  934.0
  641.0
 1000.0
 1000.0
  988.0
  382.00000000000006
  321.0
  546.0
  302.0
  236.0
  676.0
 1000.0
  408.0
  589.0

In [18]:
print_routes_and_costs(vrp_x, c_mat)

[0, 7, 12, 5, 0]	22.3
[0, 8, 14, 15, 6, 13, 0]	42.900000000000006
[0, 10, 1, 2, 0]	32.5
[0, 11, 9, 3, 4, 0]	31.5


# VRP with logical constraints

In [19]:
using Gurobi

In [20]:
vrp_log = Model(Gurobi.Optimizer)
@variable(vrp_log, x[1:n+1, 1:n+1], Bin)
@variable(vrp_log, s[1:n+1] >= 0)
@objective(vrp_log, Min, transpose(vec(c_mat))*vec(x))
for i = 1: n+1
    @constraint(vrp_log, x[i,i] == 0)
end
@constraint(vrp_log, sum(x[1, 2:n+1]) <= M)
@constraint(vrp_log, sum(x[2:n+1, 1]) <= M)
for i = 2:n+1
    @constraint(vrp_log, sum(x[i, 1:n+1]) == 1)
end
for j = 2:n+1
    @constraint(vrp_log, sum(x[1:n+1, j]) == 1)
end
@constraint(vrp_log, s[1] == 0)
for i = 2:n+1
    @constraint(vrp_log, s[i] <= Q)
end
for i = 1:n+1
    for j = 2:n+1
        @constraint(vrp_log, x[i,j] => {s[j] >= s[i] + w_cor[j]})
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-27


In [21]:
print(vrp_log)

In [22]:
optimize!(vrp_log)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 64 rows, 272 columns and 542 nonzeros
Model fingerprint: 0x1928b9d6
Model has 240 general constraints
Variable types: 16 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
  GenCon rhs range [1e+01, 5e+02]
  GenCon coe range [1e+00, 1e+00]
Presolve added 178 rows and 0 columns
Presolve removed 0 rows and 17 columns
Presolve time: 0.04s
Presolved: 242 rows, 255 columns, 1458 nonzeros
Variable types: 15 continuous, 240 integer (240 binary)
Found heuristic solution: objective 485.0000000

Root relaxation: objective 9.064480e+01, 63 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

In [23]:
solution_summary(vrp_log)

* Solver : Gurobi

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution
  Objective value      : 129.2
  Objective bound      : 129.2
  Dual objective value : 129.2

* Work counters
  Solve time (sec)   : 4.39535
  Barrier iterations : 0


In [25]:
vrp_log_x = !=(0).(value.(x))

16×16 BitMatrix:
 0  0  0  0  0  0  0  1  1  0  1  1  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0

In [26]:
vrp_log_s = value.(s)

16-element Vector{Float64}:
   0.0
 439.0
 934.0000000000001
 534.0
 893.0
 823.9999999999999
 753.0000000000001
 382.0
 321.0
 439.0
 302.0
 236.0
 675.9999999999999
 765.0000000000001
 408.00000000000017
 589.0000000000001

In [27]:
print_routes_and_costs(vrp_log_x, c_mat)

[0, 7, 12, 5, 0]	22.3
[0, 8, 14, 15, 6, 13, 0]	42.900000000000006
[0, 10, 1, 2, 0]	32.5
[0, 11, 9, 3, 4, 0]	31.5
